Este colab fue desarrollado por Arnold Charry Armero.

# General Cars

Durante los próximos dos meses, General Cars deberá cumplir
(a tiempo) las siguientes demandas para camiones y automóviles: mes
1: 400 camiones, 800 automóviles; mes 2: 300 camiones, 300 automóviles.
Durante cada mes se pueden producir como máximo 1.000 vehículos.
Cada camión utiliza 2 toneladas de acero y cada automóvil utiliza 1 tonelada de
acero. Durante el mes 1, el acero cuesta $\$ 400$ por tonelada; durante
mes 2, el acero cuesta $\$ 600$ por tonelada. Como máximo, 1.500 toneladas de
El acero se puede comprar cada mes (el acero sólo se puede utilizar durante el mes en que se compra). Al principio
del mes 1, hay 100 camiones y 200 automóviles en inventario. En el
Al final de cada mes, un costo de mantenimiento de $\$ 150$ por vehículo es
juzgado. Cada automóvil obtiene 20 mpg y cada camión obtiene
10 millas por galón. Durante cada mes, los vehículos producidos por la
La empresa debe promediar al menos 16 mpg. Formule un LP para
Cumplir con los requisitos de demanda y kilometraje como mínimo.
costo (incluye costos de acero y costos de mantenimiento).

#Conjuntos

$ t = \text{Periodo indexado por } t$

# Parámetros

$CT_{t} = \textrm{Costo de utilizar toneladas } \textrm{en el tiempo } t \textrm{ para los camiones }$

$CP_{t} = \textrm{Costo de utilizar toneladas } \textrm{en el tiempo } t \textrm{ para los carros }$

$ CI_{t} = \textrm{Costo de mantener en inventario una unidad } t $

$ Q_{t} = \textrm{Cantidad máxima de vehículos que se pueden producir en el tiempo  } t $

$ MP_{t} = \textrm{Materia prima indexada por } t $

$ a_{t} = \textrm{Demanda de camiones en el tiempo }  t$

$ b_{t} = \textrm{Demanda de carros en el tiempo }  t$

$ G_{t} = \textrm{Grado de mpg en el periodo }  t $

## Variables de decisión

$ T_{i} = \textrm{Unidades a producir de camiones en el periodo } t $

$ C_{i} = \textrm{Unidades a producir de carros en el periodo } t $

$ I_{t} = \textrm{Unidades a mantener en inventario en el periodo para los camiones en } t$

$ J_{t} = \textrm{Unidades a mantener en inventario en el periodo para los carros en } t$

## Función objetivo

$$ \textrm{Min } z = \sum_{t = 1}^{n}T_{t} CT_{t} +\sum_{t = 1}^{n} C_{t} CP_{t} + \sum_{t = 1}^{n} I_{t}CL_{t} + J_{t}CL_{t}$$

# Restricciones

Se debe satisfacer la demanda en cada periodo $t$

$$ I_{t} = I_{t-1} + T_{t} - a_{t}, \forall t $$

$$ J_{t} = J_{t-1} + C_{t} - b_{t}, \forall t $$

Se debe de tener cierto grado de mpg en los vehículos,

$$ 10T_{t} + 20C_{t} \geq 16 (T_{t} + C_{t}), \forall t$$

Se tiene un máximo de materia prima disponible,

$$ 2T_{t} + C_{t} \leq MP_{t}, \forall t $$

Se pueden producir como máximo 1000 vehículos,

$$ T_{t} + C_{t} \leq Q_{t}, \forall t $$

In [3]:
#Se instala la libreria
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 34.0 MB/s eta 0:00:00


In [4]:
#Se importa la libreria
from pulp import *

In [2]:
#Conjuntos
periodos = range(3)

In [5]:
#Parámetros

CT = [800, 1200]

CP = [400, 600]

CI = [150]

Q = [1000, 1000]

MP = [1500, 1500]

a = [400, 300]

b = [800, 300]

G = [10, 20]

In [11]:
#Se crea la instancia del modelo
model = LpProblem(name = "General_Cars", sense = LpMinimize)

#Inicialización de las variables de decisión
T = LpVariable.dicts("T", (periodos), 0, None, LpInteger)
C = LpVariable.dicts("C", (periodos), 0, None, LpInteger)
I = LpVariable.dicts("I", (periodos), 0, None, LpInteger)
J = LpVariable.dicts("J", (periodos), 0, None, LpInteger)

#Se agrega la función objetivo
model += lpSum([T[t] * CT[t-1] for t in range(1, 3)]) + lpSum([C[t] * CP[t-1] for t in range(1, 3)]) + lpSum([I[t] * CI[0] for t in range(1, 3)]) + lpSum([J[t] * CI[0] for t in range(1, 3)])

#Restricciones

#Debe haber un inventario inicial
model += I[0] == 100
model += J[0] == 200
model += T[0] == 0
model += C[0] == 0

#Tiene que satisfacerse el inventario
for t in range(1, 3):
  model += I[t-1] +  T[t] - a[t-1] == I[t]

for t in range(1, 3):
  model += J[t-1] + C[t] - b[t-1] == J[t]

#Debe tener cierto grado de mpg
for t in range(1, 3):
  model += 10 * T[t] + 20 * C[t] >= 16 * (T[t] + C[t])

#No se pueden producir más de 1000 vehículos
for t in range(1, 3):
  model += T[t] + C[t] <= Q[t-1]

#No se pueden producir más de la materia prima disponible
for t in range(1, 3):
  model += 2 * T[t] + C[t] <= MP[t-1]

#Se resuelve el modelo
model.solve()

#Se imprime el status del problema
print("Status", LpStatus[model.status])

#Se imprimen las variables
for v in model.variables():
  print(v.name, "=", v.varValue)

#Se imprime la función objetivo
print("Costo total = $", value(model.objective))

Status Optimal
C_0 = 0.0
C_1 = 600.0
C_2 = 300.0
I_0 = 100.0
I_1 = 100.0
I_2 = 0.0
J_0 = 200.0
J_1 = 0.0
J_2 = 0.0
T_0 = 0.0
T_1 = 400.0
T_2 = 200.0
Costo total = $ 995000.0
